<a href="https://colab.research.google.com/github/rakeshsiri/data-x-2-lab/blob/master/Report_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Akbar Surani

Anmol Dureha

Pradeep Kumar

Rakesh Siri

.............

Final Project Report - Part 1

Data-X 2 

12th December, 2020

Importing required libraries

In [0]:
import pandas as pd
import numpy as np
import re
import os
# from tqdm import tqdm

In [0]:
# import dask
# from dask import delayed
# from dask.distributed import Client
# client = Client(n_workers=8)
# client

In [0]:
re.compile('[A-Z]{2,}|\(.*\)').split('rakesh SIRI is SIRI (asdas asd sdsa) here')

['rakesh ', ' is ', ' ', ' here']

In [0]:
def qualify_sentence(line):
    line = line.strip()
    if re.compile('[A-za-z]{3,}').search(line):
        return True
    return False

def script_to_dialogs(movie_file, genre):
    path = f'Data/imsdb_scenes_dialogs_nov_2015/dialogs/{genre}/{movie_file}'
    with open(path,'r') as file:
        data = " ".join(line.strip() for line in file)

    regex = '[A-Z]{2,}'
    data = re.compile(regex).split(data)
    data = [d.strip() for d in data if qualify_sentence(d)]
    pd.DataFrame(data).to_csv(f'dialogs/{movie_file}',header=False,index=False)
    return 1

In [0]:
genre = os.listdir('Data/imsdb_scenes_dialogs_nov_2015/dialogs/')

In [0]:
done = 0
for g in genre:
#     print(g,'------------------------------')
    for title in tqdm(os.listdir(f'Data/imsdb_scenes_dialogs_nov_2015/dialogs/{g}/')):
#         print('      ',g)
        done += script_to_dialogs(title,g)
done

100%|██████████| 28/28 [00:00<00:00, 156.77it/s]


2498

In [0]:
list(pd.read_csv(f"dialogs/{os.listdir('dialogs/')[np.random.randint(0,961,1)[0]]}").iloc[:,0])

['What was that, fellas?  2.',
 'Clear-air turbulence, is all. Nothing',
 "But nothing you'd want to toss a  third-down pass through either --  (Grinning back at",
 ')  Can I have an autograph for my kid,  Mr. Gordon?',
 'Sure -- my pleasure.',
 "Wow. Call Westchester Approach, see  what they've got.",
 "I sure hope we don't have to turn  back. I mean this is first day of  training camp, I wouldn't want to be",
 'Seat-belt time.',
 "Maybe it'd be smoother if you went",
 'Will you',
 "'s head",
 ")  Look, Mister Flash Gordon, they have  their hands full -- just let 'em  drive.  3.",
 "When you're nervous, it can help to  chew on something",
 'Thanks a lot -- I look dumb enough  to take candy from a stranger?',
 "I'm not a stranger exactly -- You  know my name.",
 "Who doesn't. Number one draft pick,  cover of",
 "mag -- what'd the",
 'sign you for, eighty-nine  million? Big deal!',
 'Of course.',
 "God, I hate flying -- I'm Dale Arden  it's crazy of me to hate flying --  I'm a travel ag

In [0]:
a = [pd.read_csv(f'dialogs/{i}') for i in os.listdir('dialogs/')]

In [0]:
all_lines = np.concatenate(a)
all_lines = pd.DataFrame(all_lines.reshape(1,-1)[0])

In [0]:
all_lines.shape

(771808, 1)

In [0]:
all_lines.to_csv('all_lines.txt',index=False)

## **TextRank Algorithm**


Import the required libraries

In [0]:
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt') # one time execution
import re
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx

[nltk_data] Downloading package punkt to /home/rsiri/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


We have to get rid of stopwords which occurs more freqent in the text such as is, am, the, of, in, etc. We first downloaded the stopwords from nltk and from nltk we get what are the most common words in english language

In [0]:
nltk.download('stopwords')# one time execution

[nltk_data] Downloading package stopwords to /home/rsiri/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

We defined a function to remove the stopwords from the sentence

In [0]:
# function to remove stopwords
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

We have downloaded the GloVe embedding to vectorise the sentence

In [0]:
# download pretrained GloVe word embeddings
! wget http://nlp.stanford.edu/data/glove.6B.zip

--2020-01-13 02:49:47--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-01-13 02:49:49--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-01-13 02:49:50--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [0]:
! unzip glove*.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


We have created a function which takes the movie file name and and returns ranking of the sentence which is most relevent for summary generation. We have achieved this by implementing the following steps:


*   First we have cleaned the data by removing punctuations, numbers and special characters and then we coverted the entire sentence into lower case character. We also removed the stop words from the sentence after that.
*   We find vector representation (word embeddings) for each and every sentence
* We then computed the similarities between sentence vectors 

* The similarity matrix is then converted into a graph, with sentences as vertices and similarity scores between the sentences as edges, for sentence rank calculation

We then sorted the sentences occording to the ranking in descending order

In [0]:
def textrank(filename):
    # remove punctuations, numbers and special characters
    clean_sentences = pd.read_csv(f'dialogs/{filename}_dialog.txt').iloc[:,0].str.replace("[^a-zA-Z]", " ")

    # make alphabets lowercase
    clean_sentences = [s.lower() for s in clean_sentences]

    # remove stopwords from the sentences
    clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
    print('Clean words:',clean_sentences[:5])
    
    # Extract word vectors
    word_embeddings = {}
    f = open('glove.6B.100d.txt', encoding='utf-8')
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        word_embeddings[word] = coefs
    f.close()
    
    sentence_vectors = []
    for i in clean_sentences:
        if len(i) != 0:
            v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
        else:
            v = np.zeros((100,))
        sentence_vectors.append(v)
    print(f'no.of sentences: {len(sentence_vectors)}')
    
    sentence_vectors = np.array(sentence_vectors)
    
    #The next step is to find similarities among the sentences. We will use cosine similarity to find similarity 
    #between a pair of sentences. Let's create a similarity matrix for this task by
    #populating it with cosine similarities of the sentences.
    sim_mat = cosine_similarity(sentence_vectors, sentence_vectors)
    
    nx_graph = nx.from_numpy_array(sim_mat)
    scores = nx.pagerank(nx_graph)
    
    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(clean_sentences)), reverse=True)
    return ranked_sentences
   

In [0]:
ranked_sentences = textrank('afewgoodmen')

Clean words: ['quietly lucky us willy', 'requesting captain like request attorney assigned rep like request assigned represent stops assigned represent good jo confidence inspiring', 'jo come', 'thank sir', 'captain west lt commander galloway jo know mike lawrence']
no.of sentences: 1476


In [0]:
sn = 5

# Generate summary
for i in range(sn):
    print(i,':',ranked_sentences[i][1])

0 : absolutely answer first damn clue maybe early morning riser liked pack nq maybe friends educated man afraid speak intelligently travel habits william santiago know set leave base really questions called answer phone calls footlockers please tell got something lieutenant please tell ace sleeve two marines trial lives please tell lawyer pinned hopes phone bill beat questions counselor
1 : continuing call president yet maybe consider second maybe spit balling maybe officers responsibility train santiago maybe officers responsibility country see men women charged security trained professionals yes certain read somewhere thinking suggestion transferring santiago expeditious certainly painless might manner speaking american way santiago stays gonna train lad charge jon santiago make next fitness report gonna blame gonna kill
2 : pause gonna give twelve years go getting trouble tomorrow know platoon commander lt jonathan kendrick meeting men specifically told touch santiago
3 : sir joined

In [0]:
ranked_sentences = textrank('fightclub')

Clean words: ['v people always asking know tyler durden', 'one minute looking window beginning ground zero maybe say words mark occasion', 'ann iinn ff nnyin', 'v gun barrel teeth speak vowels', 'still distorted think anything']
no.of sentences: 1301


We have taken first few lines to represent the summary of the movie

In [0]:
sn = 5

# Generate summary
for i in range(sn):
    print(i,':',ranked_sentences[i][1])

0 : v even could tell someone good fight talking man
1 : happy asked one thing one simple promise look done
2 : want die without scars much really know never fight come person ever asked
3 : well let see thing
4 : v plan maybe see till hit eyes pause started make sense tyler sort way fear distractions ability let matter truly slide


## **Bert Extractive Summarizer**


Importing required library

In [0]:
import os
import pandas as pd

We have imported Bert Extractive Summarizer, which is a library that gives us access to a pre-trained BERT-based text summarization model

In [0]:
from summarizer import Summarizer
model = Summarizer()

Summarizer works by first embedding the sentences, then running a clustering algorithm, finding the sentences that are closest to the cluster's centroids.

In [0]:
Summarizer?

Init signature:
Summarizer(
    model:str='bert-large-uncased',
    custom_model:transformers.modeling_utils.PreTrainedModel=None,
    custom_tokenizer:transformers.tokenization_utils.PreTrainedTokenizer=None,
    hidden:int=-2,
    reduce_option:str='mean',
    greedyness:float=0.45,
    language=<class 'spacy.lang.en.English'>,
    random_state:int=12345,
)
Docstring:      Deprecated for naming sake.
Init docstring:
This is the main Bert Summarizer class.

:param model: This parameter is associated with the inherit string parameters from the transformers library.
:param custom_model: If you have a pre-trained model, you can add the model class here.
:param custom_tokenizer: If you have a custom tokenizer, you can add the tokenizer here.
:param hidden: This signifies which layer of the BERT model you would like to use as embeddings.
:param reduce_option: Given the output of the bert model, this param determines how you want to reduce results.
:param greedyness: associated with the neu

In [0]:
sample_text = '''
When Mr. Bilbo Baggins of Bag End announced that he would shortly be
celebrating his eleventy-first birthday with a party of special
magnificence, there was much talk and excitement in Hobbiton.
Bilbo was very rich and very peculiar, and had been the
wonder of the Shire for sixty years, ever since his remarkable
disappearance and unexpected return. The riches he had brought back
from his travels had now become a local legend, and it was popularly
believed, whatever the old folk might say, that the Hill at Bag End
was full of tunnels stuffed with treasure. And if that was not enough
for fame, there was also his prolonged vigour to marvel at. Time wore
on, but it seemed to have little effect on Mr. Baggins. At ninety he
was much the same as at fifty. At ninety-nine they began to call him
well-preserved; but unchanged would have been nearer the mark. There
were some that shook their heads and thought this was too much of a
good thing; it seemed unfair that anyone should possess (apparently)
perpetual youth as well as (reputedly) inexhaustible wealth.
"It will have to be paid for," they said. "It isn"t natural,
and trouble will come of it!"
'''
model(sample_text)

'When Mr. Bilbo Baggins of Bag End announced that he would shortly be\ncelebrating his eleventy-first birthday with a party of special\nmagnificence, there was much talk and excitement in Hobbiton. Bilbo was very rich and very peculiar, and had been the\nwonder of the Shire for sixty years, ever since his remarkable\ndisappearance and unexpected return. There\nwere some that shook their heads and thought this was too much of a\ngood thing; it seemed unfair that anyone should possess (apparently)\nperpetual youth as well as (reputedly) inexhaustible wealth.'

In [0]:
files = pd.Series(os.listdir('dialogs/'))

Finding the movies that contains a word

In [0]:
movie_name_like = 'up'
files[files.str.contains(movie_name_like)].str.replace('_dialog.txt','')

3                 upintheair
69             backupplanthe
166                 ducksoup
189                 croupier
207                supergirl
250                   tincup
254              kungfupanda
318                   super8
355                       up
458                 aptpupil
502              changeupthe
583                 superbad
686                knockedup
766    iloveyouphillipmorris
772          crazystupidlove
dtype: object

We have created a function that will return the summary of movie obtained from Bert Extractive Summarizer

In [0]:
def summary(movie_name):
    body = ""
    with open(f'dialogs/{movie_name}_dialog.txt', encoding="utf-8") as f:
        body = f.readlines()
        body = "\n".join(body)
    result = model(body, min_length=10,max_length=50)
    print(result)

In [0]:
summary('afewgoodmen')

Good, Jo, that's  confidence inspiring." "Thank you, sir." Someone who  can really look into this. (beat)  Myself." "Certainly, sir." She disposed of three cases in two  years. She's not cut out for litigation. All passion, no street smarts. "Nothing to be sorry about, Sherby. "(calling back)  I don't think so, Danny." I will get it. "Guantanamo Bay, Cuba. Commander Joanne  Galloway. She's the one who had 'em  brought up here. I'm lead counsel. This is Sam  Weinberg. Going on nine months now. (beat)  I see. "Commander, Lt. They'll need a lawyer. You with me?" Twelve years. "My job is to make sure you do your  job. Read the letters. "Sorry, I always forget that." "(V.O.)  Dear Sir,"

About being marines. Who the fuck is William T.  Santiago. "You're appalled? I've got a  better idea. Let's -- on second  thought -- Windward. (calling out)  Tom!" "Wait a minute, Tom." "(continuing)  Don't call the President just yet. "(continuing)  Matthew, sit, please." "(continuing)  Matthew, siddown. "

In [0]:
summary('up')

"(V.O.)  Why, our subject today: Charles  Muntz!" """Adventure is out there!""" But what's this? And I will not come back until I do! (V.O.)  He hurdles the Grand Canyon! (V.O.)  He... goes around Mount Everest! Is there nothing he cannot do? Let's take  her up to 26,000 feet! There's something down there! I will bring it back for science. (O.S.)  No time! Do you think you got what  it takes? "Hey, I don't bite." "You and me, we're in a club now." There it is. Thought you might need a little  cheerin' up. "Charles Muntz, explorer. You can take us there  in a blimp! "You know, you don't talk very much. "Quite a sight, huh Ellie? (noticing mailbox)  Uhp, mail's here. Oh brother. "Uh, I take that as a no, then." he's out to get  your house!" My name is  Russell.""" "Thank you, but I don't need any help! These are my Wilderness  Explorer badges. "It's gonna be great! I'm elderly and  infirm. "I'll find it, Mr. Fredricksen!" If you go past --

Two blocks down! "Hey sir, I..."

I don't want 

**References:**


1.   
https://pypi.org/project/bert-extractive-summarizer/
2.   https://www.analyticsvidhya.com/blog/2018/11/introduction-text-summarization-textrank-python/


### After this, we pivoted from our project and started solving another problem. So please find Part 2 of the report at https://colab.research.google.com/drive/19ijDj96QY2XXTJ14gxaYkxhwy_fcZw7i?authuser=2#scrollTo=QsEgNA-YkiFx